In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import missingno as msno
from statsmodels.graphics.mosaicplot import mosaic
from pandas_profiling import ProfileReport
import os

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
plt.rcParams["figure.figsize"] = (8, 6)
plt.rcParams["figure.dpi"] = 100
plt.style.use("bmh")

In [ ]:
os.chdir("..")

# Quickstart
1. [Import danych](#Import-danych-i-podstawowe-informacje-o-zbiorze)
2. [Wykresy](#Wykresy-i-wnioski-przyblizajace-zbior-danych)
3. [Potencjalne pytania](#Pytania,-ktore-moga-potencjalnie-naprowadzic-na-cechy,-ktore-maja-wieksze-znaczenie-dla-predykcji-targetu)
4. [Key takeaways](#Key-takeaways)

# Import danych i podstawowe informacje o zbiorze

In [ ]:
df = pd.read_csv("data/bank-additional-full.csv", sep=";")

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.nunique()

In [ ]:
df = df.rename(columns={"y": "target"})
df["target"] = df.target.map({"yes": 1, "no": 0})
df = df.astype({"target": "int0"})

#  Wykresy i wnioski przyblizajace zbior danych

In [ ]:
fig, axs = plt.subplots(10, figsize=(15, 24))

for ind, feature in enumerate(df.select_dtypes("object")):
    sns.countplot(data=df, x=feature, hue="target", ax=axs[ind])

plt.subplots_adjust(hspace=0.3)

### Wnioski
- Najwiecej rozmow przeprowadza sie w okresie letnim.
- Znaczna wiekszosc osob juz bralo wlasna pozyczke.
- Podobna ilosc osob maja do splacenia kredyt mieszkaniowy, do osob ktore takowego kredytu nie posiadaja w ogole.
- Pomijalna ilosc osob, ktore maja do splacenia wlasny kredyt.
- Wiekszosc osob ma wyksztalcenie wyzsze, lub srednie.
- Wiecej niz polowa osob jest zareczona. Co wazniejsze, porownujac target dla osob, bedacych singlami, do osob bedacych zareczonych, to mimo ze singli jest dwa razy mniej, to ilosc osob, dla ktorych _target_ jest rowny __1__, jest podobna.
- Znaczna wiekszosc osob ze zbioru danych to; _technician_, _admin_, lub _blue collar_.

#### Kolumna _Day of week_ jest bezuzyteczna, gdyz dystrybucja obserwacji jest taka sama miedzy roznymi wartosciami w tym polu

In [ ]:
df = df.drop("day_of_week", axis=1)

In [ ]:
fig, axs = plt.subplots(11, figsize=(18, 24))

for ind, feature in enumerate(df.select_dtypes(exclude="object")):
    sns.histplot(
        data=df,
        x=feature,
        hue="target",
        ax=axs[ind],
        bins=(2 * int(np.sqrt(df[feature].nunique()))),
    )

plt.subplots_adjust(hspace=0.4)

### Wnioski
- tu
- wypisz
- wnioski
- dla obserwacji, ktore moga miec znaczenie

In [ ]:
plt.figure(
    dpi=300,
)
subset = df[~df["education"].isin(["unknown", "illiterate"])]
subset = df[~df["job"].isin(["unknown", "housemaid"])]
mosaic(
    subset,
    ["education", "job"],
    labelizer=lambda x: "" + x[1][:4] + "--" + x[0][:4],
    label_rotation=[45, 0],
    gap=0.02,
)
plt.show()

#### Wnioski
- Tutaj tylko mozna zobrazowac ilosc konkretnych grup, wiec mozna napisac jakie grupy sa najliczniejsze
- dla tych yez grup nalezaloby zapewne zrobic plot, ktory pokazuje ze np. osob bedacych _adminem_ o wyksztalceniu 
- wyzszym, jest tyle osob, co skusilo sie na depozyt, a tyle sie nei skusilo, zrobic tak dla __3__ grup i bajka
- mozesz to zrobic crosstabem (pd.crosstab)

In [ ]:
sns.pairplot(
    df.sample(1000).select_dtypes(exclude="object"),
    hue="target",
    markers="+",
    diag_kind="kde",
)

#### Wnioski
- tu wnioski
- dla tych par, ktore maja jakis ciekawy rozklad

In [ ]:
sns.catplot(
    hue="target",
    x="poutcome",
    y="campaign",
    kind="point",
    data=df,
)
sns.catplot(
    hue="target",
    x="poutcome",
    y="cons.conf.idx",
    kind="point",
    data=df,
)

### Wnioski
- tu wnioski

In [ ]:
sns.scatterplot(data=df, x="age", y="campaign", hue="target");

### Wnioski
- tu wnioski

In [ ]:
sns.heatmap(
    data=df.select_dtypes(exclude="object").corr(),
);

#### Wnioski
- Silna korelacja cechy _duration_ z _target'em_
- Warto zwrocic uwage na silna korelacje miedzy wartosciami z grupy cech _social and economic context attributes_.
- Wzglednie silna korelacja miedzy _pdays_, a cechami _nr\_employed_, oraz silnie ujemna z _previous_.

In [ ]:
msno.bar(df);

#### Wnioski
Zbior danych, nie zawiera zadnych brakow.

#### Usuniecie wierszy zduplikowanych, czyli bedacych powtorzeniem wiersza, ktory wczesniej juz wystapil (te, nie sa brane pod uwage.

In [ ]:
df = df.loc[~df.duplicated()]

In [ ]:
pr = ProfileReport(df)

##### Profile Report report jako widget (odkomentowac komorke ponizej)

In [ ]:
# pr.to_widgets()

# Pytania, ktore moga potencjalnie naprowadzic na cechy, ktore maja wieksze znaczenie dla predykcji targetu
- Zbior danych zawiera wiele kolumn bedacych tekstem. Z tego tez powodu, nalezy pochylic sie nad procesem zamiany tekstu na wartosci liczbowe za pomoca roznego typu procesow _encoding_.
- Czy kolumny zwiazane z _social and economic context attributes_ maja faktyczne znaczenie dla predykcji? Sa bowiem one zmienne w okresach kwartalnych/miesiecznych i sa niezmienne dla pewnej grupy obserwacji.
- Kolumny z wczesniej wymienionej kategorii moga miec duze znaczenie dla zbioru danych, gdyz wskazuja na stopien zatrudnienia w kraju i stope procentowa, a ta moze miec ogromne znaczenie dla checi klientow do depozytu. Sa to bardzo wazne wspolczynniki, gdyz pozwola uogolnic model, do jego szerszego przystosowania.

# Key takeaways
- Trzeba pamietac, ze _unknown_ mozna uznac za klase, lub zmapowac jako pd.NA, bo nie znamy tej wartosci. Warto przyjrzec sie poszczegolnym cechom, w ktorej mozna zaobserwowac obiekty o wartosci cech _unknown_ i sprobowac aproksymowac wartosc tej cechy, na podstawie grupy obiektow, lub tez wyrzucic te obiekty, ktore maja ceche _unknown_, jesli ilosc jej obserwacji jest znikoma na tle calego zbioru danych.
- Warto zmapowac kolumny yes/no/unkown na 1/0/-1
- wartosc __999__ w kolumnie _pdays_ oznacza, ze nie kontaktowano sie z dana osoba
- kolumna _duration_, nie powinna pod zadnym pozorem zostac uzyta do trenowania modelu, a jedynie jego walidacji
- Wskaznik _Euribor_ to srednia arytmetyczna wiborow (stop procentowych,  podawanych przez banki UE 

In [ ]:
dict_eng_to_pl = {
    "y": "target",
    "emp.var.rate": "wsk_zmien_zatrudnienia",
    "cons.price.idx": "wsk_cen_konsum",
    "cons.conf.idx": "wsk_zauf_konsum",
    "nr.employed": "liczba_pracownikow",
    "age": "wiek",
    "job": "zawod",
    "marital": "stan_cywilny",
    "education": "wyksztalcenie",
    "default": "ma_kredyt",
    "housing": "kredyt_mieszkaniowy",
    "loan": "ma_pozyczke",
    "contact": "sposob_kontaktu",
    "month": "miesiac",
    "campaign": "liczba_polaczen_aktualnej_kampanii",
    "pdays": "liczba_dni_od_ost_kontaktu",
    "previous": "liczba_polaczen_przed_aktualna_kampania",
    "poutcome": "wynik_poprzedniej_kampanii",
    "duration": "dl_polaczenia",
}
df = df.rename(columns=dict_eng_to_pl)

In [ ]:
df.to_csv("data/precleaned.csv")